# This notebook generates the pickle files based on the `.rda` files in the original repository
## Doing this in a notebook for the sake of convenience and because it only has to be done once. Keeping the original files for posterity

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from importlib import reload
import warnings
import pyreadr

In [ ]:
warnings.simplefilter("ignore")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
contestants = pyreadr.read_r(path='data/rpdr_contestants.rda')['rpdr_contestants']
contestants.season = contestants.season.str[1:].astype('int')
contestants.age = contestants.age.astype('int')
contestants.dob = pd.to_datetime(contestants.dob)
hometown_df = contestants.hometown.str.split(', ',expand=True)
hometown_df = hometown_df.rename(columns={0:'city',1:'state'}).drop(columns=[2])
hometown_df.city = hometown_df.city.astype('category')
hometown_df.state = hometown_df.state.astype('category')
contestants = pd.concat([contestants,hometown_df],axis=1).drop(columns=['hometown'])
contestants.to_pickle('data/contestants.pkl')

In [66]:
def get_episode_type(row:pd.Series)->str:
    if row.finale > 0:
        return 'Finale'
    elif row.penultimate > 0:
        return 'Penultimate'
    else:
        return 'Standard'
contep = pyreadr.read_r('data/rpdr_contep.rda')['rpdr_contep']
contep.season = contep.season.str[1:].astype(np.int8)
contep.episode = contep.episode.astype(np.int8)
contep['rank'] = contep['rank'].fillna(-1).astype(np.int8)
contep['rank'].name = 'ranking'
contep.missc = contep.missc.astype('bool')
contep.outcome = contep.outcome.astype('category')
contep.eliminated = contep.eliminated.astype('bool')
contep.participant = contep.participant.astype('bool')
contep['etype'] = contep.apply(get_episode_type,axis=1).astype('category')
contep.minichalw = contep.minichalw.fillna(-1).astype('bool')
contep.index = pd.MultiIndex.from_frame(contep[['season','episode']])
contep = contep.drop(columns=['season','episode','penultimate','finale'])
contep.to_pickle('data/contep.pkl')

In [81]:
def get_episode_type(row:pd.Series)->str:
    if row.special > 0:
        return 'Special'
    elif row.finale > 0:
        return 'Finale'
    else:
        return 'Standard'
episodes = pyreadr.read_r('data/rpdr_ep.rda')['rpdr_ep']
episodes.season = episodes.season.str[1:].astype(np.uint8)
episodes.episode = episodes.episode.astype(np.uint8)
episodes.index = pd.MultiIndex.from_frame(episodes[['season','episode']])
episodes.airdate = pd.to_datetime(episodes.airdate)
episodes.minic = episodes.minic.fillna('None').astype('string')
episodes.numqueens = episodes.numqueens.fillna(0).astype(np.uint8)

episodes['etype'] = episodes.apply(get_episode_type,axis=1)
episodes = episodes.drop(columns=['season','episode','special','finale'])
episodes.runwaytheme = episodes.runwaytheme.fillna('Not Listed').astype('string')
episodes.head()
episodes[~episodes.bottom4.isna()]

,,airdate,nickname,runwaytheme,numqueens,minic,minicw1,minicw2,minicw3,minicw4,bottom1,...,bottom3,bottom4,bottom5,bottom6,bottom7,lipsyncartist,lipsyncsong,eliminated1,eliminated2,etype
season,episode,,,,,,,,,,,,,,,,,,,,,
11,3,2019-03-14,Diva Worship,Not Listed,13,Seduce Your Way Into a Seduction Concert,Nina West,Ra’Jah O’Hara,NaN,NaN,A’keria Chanel Davenport,...,Plastique Tiara,Ra’jah O’Hara,Scarlet Envy,Shuga Cain,NaN,Jennifer Lopez,Waiting for Tonight (Hex Rector Remix),Honey Davenport,NaN,Standard
14,10,2022-03-11,Snatch Game,Holy Couture,8,Make a look out of bubble wrap,Bosco,NaN,NaN,NaN,Angeria Paris VanMichaels,...,Daya Betty,Jasmine Kennedie,Jorgeous,Lady Camden,Willow Pill,NaN,NaN,NaN,NaN,Standard
